In [ ]:
# setup en google colab
# descoméntalo si estás en dicho entorno
# !git clone https://github.com/koldLight/python-data-ull
# %cd python-data-ull

# Gráficos geoespaciales: mapas

Habitualmente necesitamos pintar datos que tienen una referencia geoespacial, por ejemplo:

* Puntos en mapas
* Rutas
* Estados, provincias, países u otras divisiones administrativas de territorios

Para hacerlo, podemos utilizar la librería `folium`, que permite utilizar [`leaflet`](https://leafletjs.com/) dentro de python.

Puedes consultar la documentación de folium [aquí](https://python-visualization.github.io/folium/).

Si te falla el `import folium` es que no tienes la librería instalada. Para hacerlo, corre desde tu terminal:

```
# si usas conda (gestor de paquetes de Anaconda)
conda update conda
conda install folium

# si usas pip
pip install folium
```

Una vez lo instalas, tendrás que reiniciar el kernel de jupyter antes de volver a probar. Puedes hacerlo desde este notebook en el menú Kernel / Restart.

Si estás usando anaconda, también puedes descomentar las siguientes líneas (quita los `#` del principio) y ejecutarlas para instalarlo. Es equivalente a abrir la terminal y lanzarlo desde ahí.

In [ ]:
#!conda update -y conda
#!conda install -y folium

In [ ]:
import folium
import pandas as pd

## Un primer mapa

Para pintar un mapa, llamamos a folium con las opciones que queramos. Por ejemplo, vamos a pintar un mapa de la zona de La Puntilla, Las Palmas de Gran Canaria

In [ ]:
puntilla = [28.1464, -15.4293]
folium.Map(location=puntilla, zoom_start=14)

Las imágenes de las calles, el mapa base, se denominan `tiles`. Podemos personalizarlo con el parámetro `tiles`. Uno que suele quedar bastante bien para pintar datos encima es `cartodbpositron`:

In [ ]:
folium.Map(location=puntilla, zoom_start=15, tiles='cartodbpositron')

Puedes echar un vistazo a un listado mayor de tiles disponibles [aquí](https://leaflet-extras.github.io/leaflet-providers/preview/).

## Marcadores

Para añadir una localización, añadimos uno o varios marcadores al mapa. Sobre ellos, podemos personalizar:

* Localización
* Textos en tooltip o popup
* Estilo (forma, color, ...)

Mira más opciones en la [documentación](https://python-visualization.github.io/folium/modules.html#folium.map.Marker).

In [ ]:
puntilla = [28.1464, -15.4293]
m = folium.Map(location=puntilla, zoom_start=15)
folium.Marker([28.1464, -15.4293], tooltip='Mercado del puerto').add_to(m)
folium.Marker([28.1509,-15.43], tooltip='Plaza del pueblo').add_to(m)
m

Vamos a añadir una serie de localizaciones a nuestro mapa. En `dat/` tenemos disponibles las localizaciones de los apartamentos de AirBnB en Madrid, descargados de [Inside AirBnB](http://insideairbnb.com/).

In [ ]:
listings = pd.read_csv('dat/listings.csv')
listings.head()

In [ ]:
# Mi mapa base sobre Madrid
map_madrid = folium.Map(location=[40.42, -3.7], zoom_start=13, tiles='cartodbpositron')

# Cojo 500 apartamentos (la muestra completa es muy grande)
listings_sample = listings.sample(500, random_state=1234)

# Los añado a mi mapa
# Voy a usar CircleMarker en lugar de Marker, porque me deja personalizar el color más fácilmente
listings_sample.apply(
    lambda row: folium.CircleMarker([row.latitude, row.longitude], tooltip=row.room_type, radius=3).add_to(map_madrid),
    axis=1
)
map_madrid

#### Ejercicio

Personaliza el mapa anterior para que:

* El tooltip (texto que sale al pasar el ratón por encima) sea el precio por noche, con el símbolo del € al final de la cantidad.
* El mapa base (tile) sea `cartodbpositronnolabels`.
* El color del marcador diferencie por tipo de habitación (`room_type`).

Nota: primero intenta hacer los dos primeros puntos, que son más sencillos. El tema del color tendrás que investigar en la documentación y/o en stackoverflow cómo conseguirlo.

## Heatmaps

Son útiles para representar densidades.

Vamos a utilizarlo para ver cuál es la concentración de apartamentos por zona.

Si consultamos la [documentación](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap), vemos que necesita un parámetro `data` que debe ser una lista de `[lat, lng]`, pero también acepta un dataframe de pandas con 2 columnas.

In [ ]:
from folium.plugins import HeatMap

In [ ]:
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')
heatmap = HeatMap(data=listings[["latitude", "longitude"]], radius=15).add_to(m)
m

## Mapa de coropletas

Son mapas sobre los que coloreamos las regiones que contiene en base a la propiedad que queremos explicar.

Vamos a pintar un mapa que represente el número de alojamientos por cada barrio.

In [ ]:
num_per_area = listings.groupby('neighbourhood').id.count().reset_index()
num_per_area = num_per_area.rename(columns={"id": "num"})
num_per_area.head()

In [ ]:
# Incializamos el mapa
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')

# Add the color for the chloropleth:
folium.Choropleth(
    geo_data='dat/neighbourhoods.geojson',
    data=num_per_area,
    columns=['neighbourhood', 'num'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlGn',
    threshold_scale=[0, 100, 500, 1000, 2000, 3000]
).add_to(m)

m

#### Ejercicio

Pinta un mapa de coropletas sobre los municipios de las Islas Canarias, coloreado según su población. Tienes dos CSVs para hacerlo:

* `dat/pob_canarias.csv`: contiene la población para cada uno de los municipios de Canarias
* `dat/mun_canarias.geojson`: contiene los polígonos de cada uno de los municipios

Opcionalmente, pon un tooltip con el nombre y la población, que se vea al pasar el ratón por encima de cada uno de los municipios.

In [ ]:
# Lectura datos población pob_canarias.csv
pob_can = pd.read_csv("dat/pob_canarias.csv", sep=";")

# Inicializo el mapa en algún punto más o menos central de las islas
map_can = folium.Map(location=[28.2916, -15.6291], zoom_start=8, tiles="cartodbpositron")

# Generar el mapa de coropletas
# Tengo que buscar cómo cruzar el dataframe con el geojson (ver en las "properties"!): features.properties.geocode
cp = folium.Choropleth(
    geo_data='dat/mun_canarias.geojson',
    data=pob_can,
    columns=['cod_ine', 'poblacion'],
    key_on='feature.properties.geocode',
    fill_color='Oranges',
).add_to(map_can)

# Podemos pintar cualquier propiedad dentro de features.properties
tooltip = foliubm.GeoJsonTooltip(
    fields=["etiqueta", "notas"],
    aliases=["Municipio", "Notas"]
).add_to(cp.geojson)

map_can

## Ejercicios con la API de citybik

Consulta la documentación de la API de [citybik](http://api.citybik.es/v2/). Quizás quieras repasar también el notebook dedicado a las APIs.

### Ejercicio: get_networks

Crea una función `get_networks()` que devuelva todas las redes de bicis disponibles.

In [ ]:
import requests

def get_networks():
    response = requests.get('http://api.citybik.es/v2/networks')
    return response.json()['networks']


### Ejercicio: pinta las estaciones en un mapa del mundo

Pinta en un mapa todas las redes de bicis disponibles en la localización en la que se encuentran. Además, añade un tooltip con el id de la red.

### Ejercicio: get_stations

Crea una función `get_stations(network_id)` que:

* Acepte un argumento `network_id` con el identificador de la red. P.e. podría tomar el valor `sitycleta-las-palmas-las-palmas-de-gran-canaria`, `bicimad`, `bicing`, `velib`, ...
* Devuelva una lista con las estaciones (y su información) de la red especificada

_Nota_: puedes concatenar cadenas de texto con `+`. P.e.: `'hola' + ' mundo'`. O usando `f'{var1} {var2}'`.

_Opcional 1_: modifica la función `get_stations(network_id)` para que, si introduces una red que no existe, devuelva una lista vacía en lugar de un error no controlado.

_Opcional 2_: modifica la función `get_stations` para que, además del parámetro actual, tenga un parámetro adicional opcional, `min_free_bikes`: número mínimo de bicis disponibles. La función deberá devolver únicamente las estaciones que cumplan la condición pasada en este parámetro. El valor por defecto de este parámetro es 0.

In [ ]:
def get_stations(network_id, min_free_bikes=0):
    response = requests.get(f'http://api.citybik.es/v2/networks/{network_id}')
    if response.status_code != 200:
        return []
    stations = response.json()['network']['stations']
    return [station for station in stations if station['free_bikes'] >= min_free_bikes]

# stations = get_stations('sitycleta-las-palmas-las-palmas-de-gran-canaria')

### Ejercicio: pintar estaciones de una red

Pinta en un mapa las estaciones de una red de bicis de tu interés, p.e. `sitycleta-las-palmas-las-palmas-de-gran-canaria` o `lovesharing-canary-islands-gran-canaria`. En el tooltip, pon el nombre de la estación.

Opcional:

* Pinta el marcador en rojo si hay menos de un 20% de las bicis disponibles en este momento, en naranja si está entre el 20% - 50%, y verde si hay más (tip: usa mejor `CircleMarker` para personalizar el color).
* Agrega al tooltip más información de tu interés.

### Ejercicio

Pinta en un heatmap las estaciones de una red de tu interés sobre su ciudad, para ver dónde hay mayor concentración de estas.